# Предсказание бойца - победителя UFC
## Расчет прибыльности
### Межгалактический хакатон 2022
Команда "Meldonium" <br/>
Состав: Вячеслав Барков, Павел Мамаев, Сергей Глуховский, Алексей Недоливко, Андрей Рем, Иван Ершов

In [ ]:
import numpy as np
from flask import Flask, request, render_template
import pickle

In [ ]:
path = 'predictions.bin'
with open(path, 'rb') as f:
	data  = pickle.load(f)

In [ ]:
def model(amount, use_model_strategy=True):
    if use_model_strategy:
        bank = 0
        for ind in data.index:
            winner, y_pred, odd1, odd2 = data.loc[ind, ['winner', 'prediction', 'f1_odds', 'f2_odds']]
            if winner == y_pred:
                if winner:
                    win_odd = odd1
                else:
                    win_odd = odd2
            else:
                win_odd = 0
            bank += amount * win_odd - amount
        return bank
    else:
        bank = 0
        for ind in data.index:
            winner, y_pred, odd1, odd2 = data.loc[ind, ['winner', 'prediction', 'f1_odds', 'f2_odds']]
            favourite = min(odd1, odd2)
            if ((favourite == odd1) and winner) or ((favourite == odd2) and ~winner):
                win_odd = favourite
            else: 
                win_odd = 0
            bank += amount*win_odd - amount
        return bank

In [ ]:
app = Flask(__name__, template_folder='/templates')

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    amount, strategy = request.form.values()
    strategy = strategy == 'YES'
    print(amount, strategy)
    output = model(int(amount), strategy)

    if output > 0:
        output = f'Вы бы могли заработать ${output}!'
    else:
        output = f'К сожалению вы бы ничего не заработали и ушли в минус :(  ${output}!'

    return render_template('index.html', prediction_text=output)

app.run(port=80)